In [1]:
!pip3 install nibabel
!pip3 install segmentation-models-3d
!pip install -U segmentation-models-pytorch
!pip install -U torchmetrics

  Obtaining dependency information for segmentation-models-3d from https://files.pythonhosted.org/packages/c2/b5/41b4792b450e5829f63b4c1585893d141f269b6405a5f0e90cca2fe69650/segmentation_models_3D-1.0.5-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.6 MB/s eta 0:00:00
  Obtaining dependency information for classification-models-3D==1.0.8 from https://files.pythonhosted.org/packages/4f/2e/0d639ceedd3e72189cd5046a099fa4a696c26a94d7c73d4e477527aa663c/classification_models_3D-1.0.8-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 79.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

# Import Libraries and Load Data

In [2]:
import numpy as np
import torch
import nibabel as nib
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


device(type='cuda', index=0)

In [3]:
import glob

data_path = '/kaggle/input/'

volumes = [file for file in glob.glob(data_path+'volume-*')]
labels = [file for file in glob.glob(data_path+'labels-*')]

# volumes.extend(glob.glob(data_path+'volume-[01].nii'))
# labels.extend(glob.glob(data_path+'labels-[01].nii'))

volumes = sorted(volumes)
labels = sorted(labels)
volumes, labels

(['/kaggle/input/volume-0.nii',
  '/kaggle/input/volume-1.nii',
  '/kaggle/input/volume-80.nii',
  '/kaggle/input/volume-81.nii',
  '/kaggle/input/volume-82.nii',
  '/kaggle/input/volume-83.nii',
  '/kaggle/input/volume-84.nii'],
 ['/kaggle/input/labels-0.nii',
  '/kaggle/input/labels-1.nii',
  '/kaggle/input/labels-80.nii',
  '/kaggle/input/labels-81.nii',
  '/kaggle/input/labels-82.nii',
  '/kaggle/input/labels-83.nii',
  '/kaggle/input/labels-84.nii'])

# Data Information

In [4]:
# data = torch.Tensor(nib.load('/content/drive/MyDrive/Datasets/volume-0.nii.gz').get_fdata()).to(device)
# print(data.shape)
# patches = pt.patchify(data, (8,8,8), step=8)
# patches.shape
# patches = data.unfold(2, 8, 8).unfold(1, 8, 8).unfold(0, 8, 8)
# patches = patches.contiguous().view(-1, 8, 8, 8)
# print(patches.shape)
# patches = patches.view(-1,1,8,8,8)
# print(patches.shape)

# Custom Dataset Class

In [5]:
import os
import shutil

class CustomDataset(Dataset):
    def __init__(self, volumes, labels, transform=None):
        self.volumes = volumes
        self.labels = labels

    def __getitem__(self, idx):
        return self.volumes[idx], self.labels[idx]

    def __len__(self):
        return len(self.volumes)

class CustomPatchDataset(Dataset):
    def __init__(self, volume_patches, label_patches, transform=None):
        self.vol_patches = volume_patches
        self.label_patches = label_patches

    def __getitem__(self, idx):
        return self.vol_patches[idx], self.label_patches[idx]

    def __len__(self):
        return len(self.vol_patches)

In [6]:
dataset = CustomDataset(volumes, labels)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Model Implementation

In [7]:
!pip3 install monai

  Obtaining dependency information for monai from https://files.pythonhosted.org/packages/08/94/e8a7ba00dd0c7ce959648b562043bd22125d65f5e519e566c822f71bc437/monai-1.3.0-202310121228-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00a 0:00:01


In [8]:
import monai
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.networks.blocks import Convolution

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=6,
    channels=[6,16,32,64,256],
    strides=(2,2,2,2),
    num_res_units=3,
    norm=Norm.BATCH,
    act='prelu',

).to(device)

# Training Model on 2 GPU (30GB) with PyTorch DataParallel

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from segmentation_models_pytorch.losses import FocalLoss as fl, DiceLoss as dl
from segmentation_models_pytorch.metrics.functional import iou_score
from torchmetrics import JaccardIndex, Dice
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

colors = ['black', 'green', 'red', 'blue', 'purple', 'orange', 'yellow']
custom_cmap = ListedColormap(colors)

# model = nn.DataParallel(model).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_obj = fl(mode='multiclass', gamma=2).to(device)
loss_dice = Dice(multiclass=True, num_classes=6).to(device)
metric_iou = JaccardIndex(num_classes=6, task="multiclass").to(device)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_iou = 0.0
    total_loss = 0.0
    total_patches_epoch = 0
    print('#'*50)
    print('-'*20, f'EPOCH {epoch}', '-'*20)

    for batch_idx, (data_path, target_path) in enumerate(dataloader):
        print('='*50)
        print(f'Working on : {data_path} {target_path}')
        print('-'*50)
        volume = torch.Tensor(nib.load(data_path[0]).get_fdata()).to(device)
        label = torch.Tensor(nib.load(target_path[0]).get_fdata()).to(device)

        # Patchify
        volume_patches = volume.unfold(2,64,64).unfold(1,64,64).unfold(0,64,64)
        label_patches = label.unfold(2,64,64).unfold(1,64,64).unfold(0,64,64)

        volume_patches = volume_patches.contiguous().view(-1,1,64,64,64)
        label_patches = label_patches.contiguous().view(-1,1,64,64,64)

        patch_dataset = CustomPatchDataset(volume_patches, label_patches)
        patch_dataloader = DataLoader(patch_dataset, batch_size=32)

        total_patches = len(patch_dataset)
        patch_loader_size = len(patch_dataloader)
        total_patches_epoch += total_patches
        print('Total Patches', total_patches, 'Dataset Loader', patch_loader_size)

        for patch_idx, (data, target) in zip(tqdm(np.arange(patch_loader_size), total=patch_loader_size), patch_dataloader):
            batch_loss = 0.0
            batch_iou = 0.0

            # for data, target in patch_dataloader:
#             data = data.expand(-1,6,-1,-1,-1)
            # target = favourable_target(target)

            optimizer.zero_grad()
            output = model(data)

            # print(output.shape, target.shape)
            loss = loss_obj(output, target.squeeze()) + loss_dice(output, target.to(torch.int).squeeze())

            loss.backward()
            optimizer.step()

            # Calculate IOU for the current batch
            pred = torch.argmax(output, dim=1)

            iou_patch = []
            iou_patch.append(metric_iou(output, target.squeeze()))

            actual_iou_patch = sum(iou_patch)/len(iou_patch)

            total_iou += actual_iou_patch
            total_loss += loss.item()

            batch_iou += actual_iou_patch
            batch_loss += loss.item()

            # Extract the center slices (32, 32) from both images
            output_cp = output.detach().clone()

            target_cpu = target.cpu().numpy()
            output_cpu = output_cp.cpu().numpy()

            output_cpu = np.argmax(output_cpu, axis=1)

            # Now, single_channel_image contains integer values from 0 to 5, corresponding to different organs.
            center_slice_actual = target_cpu.squeeze()[16, 32, :, :]
            center_slice_predicted = output_cpu[16, 32, :, :]

            # Create a figure and plot the actual and predicted single-channel images side by side
            plt.figure(figsize=(12, 5))

            plt.subplot(1, 2, 1)
            plt.imshow(center_slice_actual, cmap=custom_cmap, vmin=0, vmax=5)
            plt.title('Actual Center Slice')

            plt.subplot(1, 2, 2)
            plt.imshow(center_slice_predicted, cmap=custom_cmap, vmin=0, vmax=5)
            plt.title('Predicted Center Slice')

            plt.savefig(f'segment-p{patch_idx}-b{batch_idx}-e{epoch}.png')
            plt.close()

            avg_batch_loss = batch_loss / len(patch_dataloader)
            avg_batch_metric = batch_iou / len(patch_dataloader)
            torch.save(model.state_dict(), 'unet3d_model.pth')

            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx+1}/{len(dataloader)}] Loss: {avg_batch_loss:.4f} IOU: {avg_batch_metric:.4f}")
            print()

    # Calculate average metrics for the epoch
    avg_iou = total_iou / total_patches_epoch
    avg_loss = total_loss / total_patches_epoch

    print(f"Epoch [{epoch+1}/{num_epochs}] Avg Loss: {avg_loss:.4f} Avg IOU: {avg_iou:.4f}")

# Save the trained model if needed
torch.save(model.state_dict(), 'unet3d_model.pth')

##################################################
-------------------- EPOCH 0 --------------------
Working on : ('/kaggle/input/volume-84.nii',) ('/kaggle/input/labels-84.nii',)
--------------------------------------------------
Total Patches 704 Dataset Loader 22


  5%|▍         | 1/22 [00:07<02:37,  7.52s/it]

Epoch [1/100] Batch [1/7] Loss: 0.1122 IOU: 0.0017



  9%|▉         | 2/22 [00:08<01:18,  3.95s/it]

Epoch [1/100] Batch [1/7] Loss: 0.0896 IOU: 0.0009



...

100%|██████████| 26/26 [00:37<00:00,  1.46s/it]

Epoch [7/100] Batch [3/7] Loss: 0.0397 IOU: 0.0197

Working on : ('/kaggle/input/volume-1.nii',) ('/kaggle/input/labels-1.nii',)
--------------------------------------------------


Total Patches 64 Dataset Loader 2


 50%|█████     | 1/2 [00:01<00:01,  1.46s/it]

Epoch [7/100] Batch [4/7] Loss: 0.5717 IOU: 0.0742



In [ ]:
# del model
# del optimizer
# torch.cuda.empty_cache()
# import gc
# gc.collect()

In [ ]:
!zip data-color-monai.zip /kaggle/working/*

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
#     command = f"zip {zip_name} {path} -r"
#     result = subprocess.run(command, shell=True, capture_output=True, text=True)
#     if result.returncode != 0:
#         print("Unable to run zip command!")
#         print(result.stderr)
#         return
    display(FileLink(f'{download_file_name}.zip'))
download_file('', 'data-color-monai')